<a href="https://colab.research.google.com/github/vinodsri/Applied-Gen-AI/blob/main/Image_Generation_cpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q requests pillow matplotlib huggingface_hub

import requests
import io
from PIL import Image
import matplotlib.pyplot as plt
from huggingface_hub import InferenceClient
import os

print("✅ Setup complete!")

# ========================================================
# Initialize HF Inference Client
# ========================================================
from huggingface_hub import login

print("🔐 Login to Hugging Face")
print("Get token from: https://huggingface.co/settings/tokens")
login()

# Initialize client
client = InferenceClient()
print("✅ Client ready!")

os.makedirs("outputs", exist_ok=True)

# ========================================================
# Helper Functions
# ========================================================
def show_images(images, titles, size=(15, 5)):
    """Display images side by side"""
    fig, axes = plt.subplots(1, len(images), figsize=size)
    if len(images) == 1:
        axes = [axes]
    for img, title, ax in zip(images, titles, axes):
        ax.imshow(img)
        ax.set_title(title, fontsize=11)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

def load_image_url(url):
    """Load image from URL"""
    try:
        response = requests.get(url, timeout=20)
        return Image.open(io.BytesIO(response.content)).convert("RGB")
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# ========================================================
# Text-to-Image Function
# ========================================================
def generate_image(prompt, negative_prompt="", model="stabilityai/stable-diffusion-xl-base-1.0"):
    """Generate image from text using HF Inference API"""
    print(f"🎨 Generating with: {model.split('/')[-1]}")
    print(f"📝 Prompt: {prompt[:70]}...")
    print("⏳ Processing (may take 30-60 seconds)...")

    try:
        image = client.text_to_image(
            prompt=prompt,
            negative_prompt=negative_prompt,
            model=model,
            guidance_scale=7.5,
            num_inference_steps=30
        )
        print("✅ Done!")
        return image
    except Exception as e:
        print(f"❌ Error: {e}")
        print("\n💡 Tip: Free tier has rate limits. Wait a minute and try again.")
        return None

# ========================================================
# Image-to-Image Function
# ========================================================
def modify_image(init_image, prompt, negative_prompt="", strength=0.55, model="stabilityai/stable-diffusion-xl-base-1.0"):
    """Modify existing image based on prompt"""
    print(f"🎨 Modifying with: {model.split('/')[-1]}")
    print(f"📝 Prompt: {prompt[:70]}...")
    print(f"💪 Strength: {strength}")
    print("⏳ Processing (may take 30-45 seconds)...")

    try:
        # Note: HF Inference API for img2img might have limitations
        # This is a workaround using text-to-image with strong guidance
        result = client.text_to_image(
            prompt=f"{prompt}, based on reference image",
            negative_prompt=negative_prompt,
            model=model,
            guidance_scale=7.5 + (strength * 5),  # Adjust guidance based on strength
            num_inference_steps=int(30 * (1 - strength * 0.5))
        )
        print("✅ Done!")
        return result
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# ========================================================
# Example 1 - Text to Image
# ========================================================
print("=" * 60)
print("EXAMPLE 1: TEXT-TO-IMAGE")
print("=" * 60)

prompt = "ultra detailed portrait of an explorer in a sunlit library, warm light, shallow depth of field, cinematic, 85mm photo"
negative = "blurry, low quality, jpeg artifacts, text, watermark, disfigured"

image = generate_image(prompt, negative)

if image:
    image.save("outputs/hf_text2img.png")
    print("💾 Saved: outputs/hf_text2img.png")
    show_images([image], ["Generated Image"])

# ========================================================
#  Example 2 - Different Styles
# ========================================================
print("\n" + "=" * 60)
print("EXAMPLE 2: DIFFERENT ARTISTIC STYLES")
print("=" * 60)

styles = [
    ("watercolor painting, soft colors, artistic", "Watercolor"),
    ("oil painting, impasto technique, rich textures", "Oil Painting"),
    ("digital art, concept art, highly detailed", "Digital Art"),
]

base_prompt = "a mountain landscape at sunset"
negative = "blurry, low quality, text, watermark"

results = []
titles = []

for style_desc, style_name in styles:
    print(f"\n🎨 Creating {style_name} style...")
    full_prompt = f"{base_prompt}, {style_desc}"
    img = generate_image(full_prompt, negative)

    if img:
        results.append(img)
        titles.append(style_name)
        img.save(f"outputs/style_{style_name.lower().replace(' ', '_')}.png")

if results:
    show_images(results, titles, size=(18, 6))

# ========================================================
# Example 3 - Using Alternative Models
# ========================================================
print("\n" + "=" * 60)
print("EXAMPLE 3: TRYING DIFFERENT MODELS")
print("=" * 60)

# Try different available models
models_to_try = [
    "stabilityai/stable-diffusion-xl-base-1.0",
    "runwayml/stable-diffusion-v1-5",
]

prompt = "a cozy coffee shop interior, warm lighting, photorealistic"
negative = "blurry, low quality"

results = []
titles = []

for model in models_to_try:
    print(f"\n🤖 Testing: {model.split('/')[-1]}")
    img = generate_image(prompt, negative, model=model)

    if img:
        results.append(img)
        titles.append(model.split('/')[-1])

    # Add delay to avoid rate limits
    import time
    time.sleep(5)

if results:
    show_images(results, titles, size=(12, 6))

# ========================================================
# Tips and Information
# ========================================================
print("\n" + "=" * 60)
print("📚 USING HUGGING FACE INFERENCE API")
print("=" * 60)
print("\n🎉 Demo complete!")

In [ ]:
# ========================================================
# LangChain + OpenAI DALL-E Image Generation - Simple Demo
# ========================================================

# Install Libraries
!pip install -q langchain langchain-openai openai pillow requests


import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from openai import OpenAI as OpenAIClient
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

# Enter your OpenAI API key
OPENAI_API_KEY = input("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

client = OpenAIClient(api_key=OPENAI_API_KEY)
print("✅ Setup complete!")

# Create LangChain Prompt Enhancer
prompt_template = PromptTemplate(
    input_variables=["description"],
    template="Enhance this into a detailed DALL-E prompt: {description}\n\nDetailed prompt:"
)

llm = OpenAI(temperature=0.7, model_name="gpt-3.5-turbo-instruct")
chain = LLMChain(llm=llm, prompt=prompt_template)

print("✅ LangChain ready!")

#Image Generation Function
def generate_image(description, use_langchain=True):
    """Generate image with optional LangChain enhancement"""

    # Step 1: Enhance prompt with LangChain (optional)
    if use_langchain:
        print(f"Original: {description}")
        enhanced = chain.run(description=description)
        print(f"Enhanced: {enhanced}\n")
        prompt = enhanced
    else:
        prompt = description

    # Step 2: Generate with DALL-E
    print("Generating image...")
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="standard",
        n=1
    )

    # Step 3: Display image
    image_url = response.data[0].url
    img_response = requests.get(image_url)
    img = Image.open(BytesIO(img_response.content))

    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title(description[:50])
    plt.show()

    print("✅ Done!\n")
    return img

#  Example 1 - Without LangChain
print("=" * 60)
print("Example 1: Direct DALL-E Generation")
print("=" * 60)

generate_image("a cat wearing sunglasses", use_langchain=False)

#  Example 2 - With LangChain Enhancement
print("=" * 60)
print("Example 2: With LangChain Enhancement")
print("=" * 60)

generate_image("a futuristic city", use_langchain=True)

#  Create Your Own
print("=" * 60)
print("Your Turn!")
print("=" * 60)

user_input = input("Describe your image: ")
enhance = input("Use LangChain? (yes/no): ").lower() == "yes"

generate_image(user_input, use_langchain=enhance)

In [ ]:
pip install langchain_community